<a href="https://colab.research.google.com/github/ancgate/hvfhv_weather_analysis/blob/main/python_scripts/hvfhv_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Libraries


In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install azure-storage-blob
!pip install google-cloud-storage
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.6/405.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 6.4 MB/s eta 0:00:00


## Importing Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import json
import re
import os
import time
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO
import pyarrow.parquet as pq

# Extracting Taxi Data from Azure

In [ ]:
AZURE_STORAGE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=sthvfhv;AccountKey=s80KlsohKqOOC6bLjRJgpmJiPqylM82RIRgREa8EAl0pttAEX9xjGZ1UFzKP6McqTpnMtcvI8mCq+AStptDIcw==;EndpointSuffix=core.windows.net"
CONTAINER_NAME = "hvfhv"
prefix = "fhvhv"  # Prefix for files

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# List and download Parquet files starting with 'fhvhv'
parquet_dfs = []  # List to hold each DataFrame

# Iterate through blobs in the container that match the prefix
for blob in container_client.list_blobs(name_starts_with=prefix):
    if blob.name.endswith(".parquet"):
        print(f"Downloading: {blob.name}")
        # Download the blob as bytes
        blob_client = container_client.get_blob_client(blob)
        parquet_data = blob_client.download_blob().readall()

        # Convert the bytes data into a DataFrame
        parquet_buffer = BytesIO(parquet_data)
        df = pq.read_table(parquet_buffer).to_pandas()

        # Append the DataFrame to the list
        parquet_dfs.append(df)
        break  # Remove this line if you want to download all Parquet files

# Concatenate all DataFrames (optional, if you want a single DataFrame)
combined_df = pd.concat(parquet_dfs, ignore_index=True)

print("Parquet files retrieved and combined successfully.")

# Extracting Data for OpenMeteo

In [ ]:
csv_buffer = StringIO()
combined_df.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)
blob_name = "weather_data.csv"
# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(csv_buffer.getvalue(), blob_type="BlockBlob", overwrite=True)

print("CSV uploaded to Azure Blob Storage successfully.")

## Cleaning Data

### Reformatting Taxi Data

### Transformation Taxi Data